In [ ]:
#this notebook establishes parameter sets that will be used by LASAM

In [ ]:
#!pip install smt

In [ ]:
import numpy as np
import random 
from datetime import datetime #used to record total runtime
# import os #needed to run system commands from python
import subprocess #needed to run system commands from python, more flexible and safer than os
from smt.sampling_methods import LHS #Latin hypercube sampling
import matplotlib.pyplot as plt
import os 

In [ ]:
theta_r_range = [0.01 ,0.15]
theta_e_range = [0.3, 0.8]
log_alpha_range =   [-3.0, -0.52] #corresponding to 0.001 and 0.3, via log_10
n_range =       [1.01, 3] #I recommend n to be no smaller than 1.01. Weird things happen with the water retention curve below that  
log_Ks_range =      [-3, 2] #corresponding to 0.001 and 100, via log_10
initial_psi_range = [10, 3000]
with_ponded_head = [-1, 1] #later, ponded head will be set to zero or nonzero based on this
field_capacity_psi_range = [10.3, 516.5]


In [ ]:
#establishes limits for Latin hypercube.

xlimits = np.array([[theta_r_range[0], theta_r_range[1]],
                    [theta_e_range[0], theta_e_range[1]],
                    [log_alpha_range[0], log_alpha_range[1]],
                    [n_range[0], n_range[1]],
                    [log_Ks_range[0], log_Ks_range[1]],
                    [initial_psi_range[0], initial_psi_range[1]],
                    [with_ponded_head[0], with_ponded_head[1]],
                    [field_capacity_psi_range[0], field_capacity_psi_range[1]],
                    ])


In [ ]:
sampling = LHS(xlimits=xlimits,random_state=2+10) #random_state is for reproducability

#this creates the Latin hypercube containing all parameter sets, where num is the number of parameter sets desired.
num = 10000
x = sampling(num)

In [ ]:
x

In [ ]:
# for h in range(num):
#     theta_r = x[h,0]
#     theta_e = x[h,1]
#     alpha = x[h,2]
#     n = x[h,3]
#     Ks = x[h,4]
#     initial_psi = x[h,5]
#     print(Ks)
#     print(n)
#     print(' ')

In [ ]:
os.chdir('../../')

In [ ]:
#testing single case where the parameters file is edited
with open('data/vG_default_params.dat', 'r') as file:
    # read a list of lines into data
    parameters_file_text = file.readlines()
    file.close()

print(parameters_file_text)


In [ ]:
parameters_file_text[0]

In [ ]:
parameters_file_text[1]

In [ ]:
parameter_set = random.randint(1,10000)
parameter_set = 0

In [ ]:
# x[parameter_set,2] = 10**x[parameter_set,2]
# x[parameter_set,4] = 10**x[parameter_set,4]
# parameters_file_text[1] = '"Clay"\t                ' + str(x[parameter_set,0]) + '\t' + str(x[parameter_set,1]) + '\t' + str(x[parameter_set,2]) + '\t' + str(x[parameter_set,3]) + '\t' + str(x[parameter_set,4]) + '\n'      

In [ ]:
parameters_file_text[1]

In [ ]:
with open('data/vG_default_params_LHS.dat', 'w') as file:
    file.writelines( parameters_file_text )
    file.close()

In [ ]:
# with open('configs/config_lasam_LHS.txt', 'r') as file:
# # read a list of lines into data
#     config_file_text = file.readlines()
#     file.close()

In [ ]:
# config_file_text

In [ ]:
# config_file_text[4]

In [ ]:
# config_file_text[4] = 'initial_psi=' + str(x[parameter_set,5]) + '[cm]\n'

In [ ]:
# with open('configs/config_lasam_LHS.txt', 'w') as file:
#     file.writelines( config_file_text )
#     file.close()

In [ ]:
# result = !./build/lasam_standalone configs/config_lasam_LHS.txt

In [ ]:
# result

In [ ]:
# result[2] == '-------------------- Simulation Summary ----------------- '

In [ ]:
# print(result)

In [ ]:
# for row in result:
#     print(row)

In [ ]:
num_runs = 0
num_successful_runs = 0
unsuccessful_parameter_sets = []
#error types are: mass balance, timeout due to infinite loop, and nonzero ponded head when its max is set to 0.
unsuccessful_parameter_sets_ponded_water = []
unsuccessful_parameter_sets_timeout = []
unsuccessful_parameter_sets_mass_bal = []
runtime_vec = []

begin_time = datetime.now() #used to record total runtime

# test_set = [9610]
# for parameter_set in test_set:
for parameter_set in range(num):
    x[parameter_set,2] = 10**x[parameter_set,2]
    x[parameter_set,4] = 10**x[parameter_set,4]
    print("parameter set:")
    print(parameter_set)
#     print("actual alpha:")
#     print(x[parameter_set,2])
#     print("actual Ks:")
#     print(x[parameter_set,4])
    with open('data/vG_default_params.dat', 'r') as file:
    # read a list of lines into data
        parameters_file_text = file.readlines()
        file.close()
    parameters_file_text[1] = '"Clay"\t                ' + str(x[parameter_set,0]) + '\t' + str(x[parameter_set,1]) + '\t' + str(x[parameter_set,2]) + '\t' + str(x[parameter_set,3]) + '\t' + str(x[parameter_set,4]) + '\n'      
    with open('data/vG_default_params_LHS_arid.dat', 'w') as file:
        file.writelines( parameters_file_text )
        file.close()
        
    with open('configs/config_lasam_LHS_Phillipsburg.txt', 'r') as file:
    # read a list of lines into data
        config_file_text = file.readlines()
        file.close()
    config_file_text[4] = 'initial_psi=' + str(x[parameter_set,5]) + '[cm]\n'
    
    if (x[parameter_set,6]<0):
        config_file_text[8] = 'ponded_depth_max=0[cm]\n'
    else:
        config_file_text[8] = 'ponded_depth_max=2[cm]\n'
    if (x[parameter_set,6]>0.5):
        config_file_text[8] = 'ponded_depth_max=5[cm]\n'
        
    config_file_text[13] = 'field_capacity_psi=' + str(x[parameter_set,7]) + '[cm]\n'
        
    with open('configs/config_lasam_LHS_Phillipsburg.txt', 'w') as file:
        file.writelines( config_file_text )
        file.close()
        
    print("running LASAM ...")
#     result = !./build/lasam_standalone configs/config_lasam_LHS_Phillipsburg.txt
#     result = !./build/lasam_standalone configs/config_lasam_LHS_Koptis_Farms.txt 
    start_time_loc = datetime.now() #used to record local runtime
    result = !./build/lasam_standalone configs/config_lasam_LHS_Phillipsburg.txt 
    end_time_loc = datetime.now() #used to record local runtime
    runtime_loc = end_time_loc - start_time_loc
    runtime_vec.append(runtime_loc.total_seconds())
    
    for row in result:
        print(row)
    print(" ")
    print(" ")
    
    run_success = False
    surface_ponded_water = 0.0;
    if (len(result) > 1):
        if (result[8][0:20]=='Surface ponded water'):
            surface_ponded_water = float(result[8][30:38])
#         if ((result[2] == '-------------------- Simulation Summary ----------------- ') & (result[8]=='Surface ponded water      =   0.0000000000 cm')):
        if ((result[2] == '-------------------- Simulation Summary ----------------- ') ):
            num_successful_runs = num_successful_runs + 1
            run_success = True

    if (run_success == False):
        unsuccessful_parameter_sets.append(parameter_set)
        print("unsuccessful parameter set:")
        print(parameter_set)
        print(" ")
        print(" ")
        print(" ")
        print(" ")
        print(" ")
#         if (surface_ponded_water>0.0):
#             unsuccessful_parameter_sets_ponded_water.append(parameter_set)
        if ('mass balance closure not possible within 100000 iterations. Timeout ' in result):
            unsuccessful_parameter_sets_timeout.append(parameter_set)
        if (len(result)>1):
            if (result[1]=='Local mass balance at this timestep... '):
                unsuccessful_parameter_sets_mass_bal.append(parameter_set)
            
            
        
    num_runs = num_runs + 1
    
end_time = datetime.now() #used to record total runtime
total_time = end_time - begin_time
print("all runs complete.")



    

In [ ]:
print("elapsed time: ")
print(total_time)
print("num_successful_runs:")
print(num_successful_runs)
print("num_runs:")
print(num_runs)
print("success rate (%): ")
print(100*num_successful_runs/num_runs)
print("unsuccessful_parameter_sets:")
print(unsuccessful_parameter_sets)
print("unsuccessful parameter sets due to erroneous ponded water: ")
print(unsuccessful_parameter_sets_ponded_water)
print("unsuccessful parameter sets due to mass balance error: ")
print(unsuccessful_parameter_sets_mass_bal)
print("unsuccessful parameter sets due to timeout (infinite loop likely): ")
print(unsuccessful_parameter_sets_timeout)

In [ ]:
plt.hist(runtime_vec)

In [ ]:
unsuccessful_parameter_sets == unsuccessful_parameter_sets_timeout

In [ ]:
# unsuccessful_parameter_sets

In [ ]:
x[unsuccessful_parameter_sets]

In [ ]:
# result[1]

In [ ]:
# unsuccessful_parameter_sets_ponded_water 

In [ ]:
# unsuccessful_parameter_sets_timeout 

In [ ]:
# unsuccessful_parameter_sets_mass_bal 

In [ ]:
# result[8][0:20]=='Surface ponded water'

In [ ]:
# result

In [ ]:
# !pwd